In [1]:
import os
import glob
import xml.etree.ElementTree as ET
import pandas as pd
import re 
from shutil import copyfile

In [70]:
parent = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2_Misha/SAFE_true_raw"
dirs = os.listdir(parent)

meta_data = []
for folder in dirs:
    xml_loc = glob.glob(parent + "/" + folder + "/*.xml")[0]
    tree = ET.parse(xml_loc)
    directory = [elem.text for elem in tree.iter() if "MASK_FILENAME" in elem.tag][0].split("/")[1]
    tile_id = directory.split("_")[1]
    filepath_partial = parent + "/" + directory + "/IMG_DATA" + "/R10m"
    filepath = glob.glob(filepath_partial + "/*.*")[0]
    filename = filepath.split("/")[-1]
    cloud_cover,no_data,unclassified = [elem.text for elem in tree.iter() if "CLOUDY_PIXEL_PERCENTAGE" in elem.tag 
             or "NODATA_PIXEL_PERCENTAGE" in elem.tag or "UNCLASSIFIED_PERCENTAGE" in elem.tag]
    meta_data.append([directory,tile_id,cloud_cover,no_data,unclassified,filename,filepath])
df = pd.DataFrame(meta_data,columns=["Directory","Tile_Id","Cloud Cover","No Data Percentage","Unclassified Percentage","Filename","Filepath"])




In [80]:
#export df to csv

In [2]:
#read in dataframe

df2 = pd.read_csv("./data/Dataframe/tile_list.csv").sort_values(by=["Tile_Id","Cloud Cover","Unclassified Percentage"],ignore_index=True)

In [3]:
df2

,Unnamed: 0,Directory,Tile_Id,Cloud Cover,No Data Percentage,Unclassified Percentage,Filename,Filepath
0,186,L2A_T33NTB_A013990_20191110T093408,T33NTB,13.929234,0.000000,3.225338,T33NTB_20191110T092119_TCI_10m.jp2,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...
1,19,L2A_T33NTB_A017851_20200806T093251,T33NTB,14.723322,0.000000,20.213281,T33NTB_20200806T092029_TCI_10m.jp2,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...
2,148,L2A_T33NTC_A023971_20200124T093439,T33NTC,20.687326,0.000000,1.377507,T33NTC_20200124T092251_TCI_10m.jp2,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...
3,146,L2A_T33NUA_A015091_20200126T091912,T33NUA,0.676950,41.920325,0.064221,T33NUA_20200126T091139_TCI_10m.jp2,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...
4,8,L2A_T33NUA_A026788_20200808T093036,T33NUA,1.135467,65.888530,2.706349,T33NUA_20200808T090601_TCI_10m.jp2,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...
...,...,...,...,...,...,...,...,...
205,177,L2A_T34NFH_A014533_20191218T084754,T34NFH,9.777429,0.000000,0.031831,T34NFH_20191218T084249_TCI_10m.jp2,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...
206,96,L2A_T34NFH_A015963_20200327T085848,T34NFH,11.286476,0.000003,2.016374,T34NFH_20200327T083559_TCI_10m.jp2,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...
207,42,L2A_T34NFH_A017107_20200615T084829,T34NFH,12.292093,0.000000,2.994645,T34NFH_20200615T083559_TCI_10m.jp2,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...
208,194,L2A_T34NFH_A021511_20190805T085826,T34NFH,14.238894,0.000000,4.557573,T34NFH_20190805T083601_TCI_10m.jp2,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...


In [4]:
df2["Tile_Id"].value_counts()

T33NYC    10
T34NDH    10
T33NXB    10
T33NYD    10
T33NWA    10
T34MCB    10
T33NVA    10
T33NUE    10
T33NZD    10
T33NVC    10
T33NWB    10
T34MBC    10
T34NFH    10
T34NEF    10
T34NDF    10
T33NVD    10
T34NFG    10
T34MBB    10
T33NUC     9
T33NUA     8
T33NUB     7
T33NUD     3
T33NTB     2
T33NTC     1
Name: Tile_Id, dtype: int64

In [5]:
base_dir = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2_Misha/SAFE_true_ordered_cloud_cover/"
layer = 1
df = df2
for index,row in df.iterrows(): 
    destination_dir = base_dir + str(layer)
    print("destination_dir:",destination_dir)
    output_file = destination_dir + "/" + row["Filename"]
    print("output_file:",output_file)
    
    # Check if directory exists
    if not os.path.isdir(destination_dir):
        os.mkdir(destination_dir)
        
    # Copy file to existing or new directory
    copyfile(row["Filepath"],output_file)
    
    # Check if Tile_Id already exists in the directory - only necessary up until the last tile
    if len(df) > index + 1:
        if df.loc[index,"Tile_Id"] == df.loc[index + 1,"Tile_Id"]:
            layer += 1
        else:
            layer = 1   

destination_dir: /Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2_Misha/SAFE_true_ordered_cloud_cover/1
output_file: /Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2_Misha/SAFE_true_ordered_cloud_cover/1/T33NTB_20191110T092119_TCI_10m.jp2
destination_dir: /Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2_Misha/SAFE_true_ordered_cloud_cover/2
output_file: /Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2_Misha/SAFE_true_ordered_cloud_cover/2/T33NTB_20200806T092029_TCI_10m.jp2
destination_dir: /Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2_Misha/SAFE_true_ordered_cloud_cover/1
output_file: /Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2_Misha/SAFE_true_ordered_cloud_cover/1/T33NTC_20200124T092251_TCI_10m.jp2
destination_dir: /Volumes/Lacie/zhenyadata/Project_Canopy_Dat

In [1]:
# for row in range(len(df5)):
#     layer = 1
#     dest_folder = f'.../layer_{layel}'
#     # Copy TCI to dest_folder
#     if not os.exists(dest_folder):
#         os.mkdir(dest_folder)

#     if row != len(df5)-1:
#         if df5.loc[row, 'tileid_new'] == df5.loc[row+1, 'tileid_new']:
#             layer += 1
#         else:
#             layer = 1

# For Finding Corresponding SCL File for each tile from SAFE_true

In [28]:
source_dir_SAFE_true = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2_Misha/SAFE_true_raw/"
source_dir_SAFE_false = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2_Misha/SAFE_false_raw/"

xml_source = glob.glob(source_dir_SAFE_true + "*/*.xml")[0]

print(os.listdir(source_dir_SAFE_false)[0],os.listdir(source_dir_SAFE_true)[0])

SAFE_false_dir = os.listdir(source_dir_SAFE_false)[0]
SAFE_true_dir = os.listdir(source_dir_SAFE_true)

34MBC,2020-08-25,0 L2A_T34MBC_A027031_20200825T092011


In [25]:
tile_id = SAFE_false_dir.split(",")[0]
tile_date = SAFE_false_dir.split(",")[1]

In [32]:
SAFE_true_dir[0].split("_")[1]

'T34MBC'